In [43]:
import pandas as pd

In [44]:
adp = pd.read_csv('adp.csv')

In [45]:
adp['team'] = adp['player'].apply(lambda x: x.split('\xa0')[-1])

In [46]:
adp['player'] = adp['player'].apply(lambda x: ' '.join(x.split('\xa0')[:-1]).upper())

In [47]:
adp['player'] = adp['player'].apply(lambda x: x \
        .replace('ALEKSANDER BARKOV JR.', 'ALEKSANDER BARKOV') \
        .replace('MITCH MARNER', 'MITCHELL MARNER') \
        .replace('MACKENZIE BLACKWOOD-DUP', 'MACKENZIE BLACKWOOD') \
        .replace('MAXIME COMTOIS', 'MAX COMTOIS'))

In [48]:
adp = adp[['yahoo', 'player']]
adp.columns = ['ADP', 'Name']

In [49]:
adp['ADP'] = pd.to_numeric(adp['ADP'], errors='coerce')

In [50]:
dfs = {}
ss = ['goalies', 'peripherals', 'pp', 'skaters']
ys = ['2122', '2021', '1920']
for s in ss:
    for y in ys:
        df = pd.read_csv(f'{s}_{y}.csv')
        df['Name'][(df['Name']=='SEBASTIAN AHO') & (df['Team']=='NYI')] = 'SEBASTIAN AHO NYI'
        dfs[(s,y)] = df

/home/lpang/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [51]:
for y in ys:
    df = dfs[('skaters',y)] \
        .merge(dfs[('peripherals',y)], how='outer', on='Name') \
        .merge(dfs[('pp',y)], how='outer', on='Name')
    df = df[['Name', 'Pos_x', 'Age_x', 'Team_x', 'GP_x', 'G_x',
            'A_x', 'PTS', 'GWG', 'TOI_x', 'SOG_x', 'Yrs', 'Hits',
            'Blocks', 'PPG', 'PPP']]
    df.columns = [x.replace('_x', '') for x in df.columns]
    dfs[('merged',y)] = df
    dfs[('goalies',y)] = dfs[('goalies',y)][['Name', 'Age', 
            'Team', 'GP', 'W', 'SO', 'SV', 'SV%', 'Yrs']]

In [52]:
joins = ['Name', 'Age', 'Yrs']
for s in ['goalies', 'merged']:
    dfs[(s,'1920')].columns = [f'{x}_1920' if x not in joins else x for x in dfs[(s,'1920')].columns]
    dfs[s,'merged'] = dfs[(s,'2122')] \
        .merge(dfs[(s,'2021')], how='left', on=joins, suffixes=('_2122','_2021')) \
        .merge(dfs[(s,'1920')], how='left', on=joins)

In [53]:
merge = pd.concat([dfs['goalies','merged'],dfs['merged','merged']])

In [54]:
merge = merge.merge(adp, on='Name', how='outer')

In [55]:
merge['Pos'] = merge['Pos_2122'].apply(lambda x: 'D' if x == 'D' else 'F' if x in ['C', 'L', 'R'] else 'G')

In [56]:
pl = pd.read_csv('placeholders.csv')
merge = pd.concat([merge, pl])

In [57]:
merge.to_csv('data.csv', index=False)

In [58]:
print(merge[merge['Age'].isna()]['Name'])

Series([], Name: Name, dtype: object)


In [59]:
print(len(merge))

1126


In [60]:
merge.columns

Index(['Name', 'Age', 'Team_2122', 'GP_2122', 'W_2122', 'SO_2122', 'SV_2122',
       'SV%_2122', 'Yrs', 'Team_2021', 'GP_2021', 'W_2021', 'SO_2021',
       'SV_2021', 'SV%_2021', 'Team_1920', 'GP_1920', 'W_1920', 'SO_1920',
       'SV_1920', 'SV%_1920', 'Pos_2122', 'G_2122', 'A_2122', 'PTS_2122',
       'GWG_2122', 'TOI_2122', 'SOG_2122', 'Hits_2122', 'Blocks_2122',
       'PPG_2122', 'PPP_2122', 'Pos_2021', 'G_2021', 'A_2021', 'PTS_2021',
       'GWG_2021', 'TOI_2021', 'SOG_2021', 'Hits_2021', 'Blocks_2021',
       'PPG_2021', 'PPP_2021', 'Pos_1920', 'G_1920', 'A_1920', 'PTS_1920',
       'GWG_1920', 'TOI_1920', 'SOG_1920', 'Hits_1920', 'Blocks_1920',
       'PPG_1920', 'PPP_1920', 'ADP', 'Pos'],
      dtype='object')

In [61]:
merge[merge['Pos'] == 'G'][['GP_2122', 'SV%_2122', 'W_2122', 'SO_2122', 'SV_2122']].describe()

,GP_2122,SV%_2122,W_2122,SO_2122,SV_2122
count,120.000000,120.000000,120.000000,120.000000,120.000000
mean,23.825000,0.890258,11.083333,1.150000,632.158333
std,20.237523,0.051325,11.381530,1.648274,563.470727
min,1.000000,0.571000,0.000000,0.000000,2.000000
25%,4.000000,0.886750,1.000000,0.000000,97.250000
50%,20.000000,0.903000,8.500000,0.000000,523.000000
75%,40.250000,0.914000,18.000000,2.000000,1026.250000
max,67.000000,0.958000,39.000000,9.000000,1962.000000


In [62]:
merge.dtypes

Name            object
Age              int64
Team_2122       object
GP_2122          int64
W_2122         float64
SO_2122        float64
SV_2122        float64
SV%_2122       float64
Yrs            float64
Team_2021       object
GP_2021        float64
W_2021         float64
SO_2021        float64
SV_2021        float64
SV%_2021       float64
Team_1920       object
GP_1920        float64
W_1920         float64
SO_1920        float64
SV_1920        float64
SV%_1920       float64
Pos_2122        object
G_2122         float64
A_2122         float64
PTS_2122       float64
GWG_2122       float64
TOI_2122        object
SOG_2122       float64
Hits_2122      float64
Blocks_2122    float64
PPG_2122       float64
PPP_2122       float64
Pos_2021        object
G_2021         float64
A_2021         float64
PTS_2021       float64
GWG_2021       float64
TOI_2021        object
SOG_2021       float64
Hits_2021      float64
Blocks_2021    float64
PPG_2021       float64
PPP_2021       float64
Pos_1920   